In [99]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr


# importlib.reload(build_db)

<module 'barchartacs.build_db' from '../barchartacs/build_db.py'>

In [62]:
SYMBOL_TO_RESEARCH = 'ES'

In [63]:
def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    # move index to date column, sort and recreate index
    df['date'] = df.index
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df['unadj_close'] = df.Close
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis=1)
    cols = df.columns.values 
    cols_dict = {c:c[0].lower() + c[1:] for c in cols}
    df = df.rename(columns = cols_dict)
    
    df['trade_date'] = df.date.apply(lambda v:int(v.strftime('%Y%m%d')))
    df['year'] = df.date.apply(lambda v: v.year)
    df['month'] = df.date.apply(lambda v: v.month)
    df['day'] = df.date.apply(lambda v: v.day)
    return df


### important global variables

In [64]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


In [65]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdvals,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig


#### get all contracts in the options database

In [78]:
pga = db_info.get_db_info()
all_contracts = pga.get_sql(f"select distinct symbol from {opttab} where symbol~'^{SYMBOL_TO_RESEARCH}'").sort_values('symbol').values.reshape(-1)
len(all_contracts)

  sec_db


82

### graph skew

In [67]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(4)
    return df_ret


In [151]:
def graph_skew(df_iv_final,do_plot=False):
    '''
    Graph skew for ONLY ONE symbol.
    If df_iv_final contains more than one symbol, we will only graph the first symbol in the DataFrames    
    '''
    # get the first symbol (which should be the only symbol)
    contract = df_iv_final.symbol.unique()[0]
    # get just that symbol's data and only days that have sufficient skew data
    dft = df_iv_final[df_iv_final.symbol==contract]
    dft_count = dft[['settle_date','symbol']].groupby('settle_date',as_index=False).count()
    valid_settle_dates = dft_count[dft_count.symbol>2].settle_date.unique()
    dft = dft[dft.settle_date.isin(valid_settle_dates)]
    dfp = create_skew_per_date_df(dft)
    
    settle_dates = sorted([c for c in dfp.columns.values if c != 'moneyness'])
    splits = list(np.arange(5,len(settle_dates),5))
    settle_date_groups = np.split(np.array(settle_dates),splits)
    ret_figs = []
    for sdg in settle_date_groups:
        sdg_sorted = [str(c) for c in sorted(sdg)]
        cols = ['moneyness']+list(sdg_sorted)
        dfp_sub = dfp[cols]
        t = f'{contract} {sdg[0]} - {sdg[-1]}' 
        f = plotly_plot(dfp_sub,x_column='moneyness',plot_title=t,y_left_label='vol skew')
        ret_figs.append(f)
        if do_plot:
            iplot(f)
    return ret_figs

In [152]:
my_list = [1, 2, 3, 4, 5, 
              6, 7] 
  
# How many elements each 
# list should have 
n = 4 
  
# using list comprehension 
final = [my_list[i * n:(i + 1) * n] for i in range((len(my_list) + n - 1) // n )]  
print (final)

[[1, 2, 3, 4], [5, 6, 7]]


#### Do the same plots as above, but using a grid

In [153]:
def graph_skew_subplots(df):
    rfs = graph_skew(df)
    n = 4   
    # using list comprehension 
    rfs_groups = [rfs[i*n:(i + 1)*n] for i in range((len(rfs) + n - 1) // n )]  
    for rfs_group in rfs_groups:
        iplot(graph_skew_subplot_quad(rfs_group))
    
def graph_skew_subplot_quad(rfs):
    '''
    Use subplots to output the results of the method graph_skew above
    '''
    rows = 2#len(rfs)//2
    f1 = tls.make_subplots(rows=rows, cols=2,  
        shared_yaxes=False, 
        subplot_titles=[rfs[i]['layout'].title for i in range(len(rfs))],
        horizontal_spacing=0.09,
        vertical_spacing=0.11,                       
        print_grid=False)

    pl_width=900
    pl_height=800 
    title = 'Skew plots<br>'

    f1['layout'].update(title=title,                                 
        font= Font(family="Open Sans, sans-serif"),
        showlegend=True,     
        hovermode='x',  
        autosize=True,       
        width=pl_width,       
        height=pl_height,
        plot_bgcolor='#EFECEA', 
        bargap=0.05,
        margin=Margin(
                      l=45,
                      r=15,
                      b=55,
                      t=50
        )
    )    

    for i in range(len(rfs)):
        x = int(i/2) + 1
        y = i % 2 + 1
        f = rfs[i]
        l = f.layout
        
        try:
            yaxis = f'yaxis{i+1}'
            xaxis = f'xaxis{i+1}'
            if i < 10:
                yaxis = yaxis.replace('1','') 
                xaxis = xaxis.replace('1','') 
            f1['layout'].update({xaxis:l.xaxis})
            f1['layout'].update({yaxis:l.yaxis})
            gname = f'chart {x,y}'#rfs[i]['layout'].title
            for d in f.data:
                data_y = f'y{i+1}'.replace('1','') 
                d['yaxis']=data_y
                d['legendgroup'] =  gname
                d['name'] = f"{gname} {d.name}"
                f1.append_trace(d,x,y)
        except Exception as e:
            print(f'graph_skew_subplots ERRORS: {str(e)}')
    return f1
# iplot(f1)


In [154]:
df_iv_final = pd.read_csv('./temp_folder/df_iv_final.csv')
df_iv_skew = pd.read_csv('./temp_folder/df_iv_skew.csv')

In [146]:
etf = 'spy'
yyyymmdd_beg = df_iv_skew.settle_date.min() 
yyyymmdd_end = df_iv_skew.settle_date.max()
dt_beg = datetime.datetime(int(str(yyyymmdd_beg)[0:4]),int(str(yyyymmdd_beg)[4:6]),int(str(yyyymmdd_beg)[6:8]))
dt_end = datetime.datetime(int(str(yyyymmdd_end)[0:4]),int(str(yyyymmdd_end)[4:6]),int(str(yyyymmdd_end)[6:8]))
df_etf = fetch_history(etf,dt_beg,dt_end)
df_etf = df_etf.rename(columns={'trade_date':'settle_date'})

In [155]:
grid_plot=True
clist = [c for c in all_contracts if (c[:2]==f'{SYMBOL_TO_RESEARCH}') & (c[-2:]=='19')]
for c in clist:
    dft = df_iv_final[df_iv_final.symbol==c]
    if len(dft)<=0:
        print(f'no data for symbol {c}')
        continue
    if grid_plot:
        graph_skew_subplots(df_iv_final[df_iv_final.symbol==c])
    else:
        rls = graph_skew(df_iv_final[df_iv_final.symbol==c],do_plot=True)

In [82]:
y = 19
c = SYMBOL_TO_RESEARCH
df_year = df_iv_skew[(df_iv_skew.symbol.str.slice(0,2)==c) & (df_iv_skew.symbol.str.slice(-2,)==str(y))]
df_year['diff'] = df_year['-0.2'] - df_year['0.2']
df_year = df_year[['settle_date','diff']]
df_year = df_year.merge(df_etf[['settle_date','close']],on='settle_date',how='inner')
iplot(plotly_plot(df_year,x_column='settle_date',y_left_label='put skew',yaxis2_cols=['close'],y_right_label=f'{etf} close'))



/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [83]:
y = 19
c = SYMBOL_TO_RESEARCH
df_year = df_iv_skew[(df_iv_skew.symbol.str.slice(0,2)==c) & (df_iv_skew.symbol.str.slice(-2,)==str(y))]
df_year = df_year[['settle_date','-0.2','0.2']]
df_year = df_year.merge(df_etf[['settle_date','close']],on='settle_date',how='inner')
iplot(plotly_plot(df_year,x_column='settle_date',yaxis2_cols=['close'],y_right_label='uso close'))

In [19]:
def get_postgres_data(contract):
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    usql = f"select * from {futtab} where symbol='{contract}';"
    dfu = pga.get_sql(usql)
    # Merge options and futures data
    df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    return df,df_expiry_dates

In [20]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    df2 = df2[df2.contract_num==2]
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



In [21]:
def get_even_moneyness_strikes(df10):
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')
    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    return df12



In [144]:
d = 5
g = 2
e = .0000001
for _ in range(100):
    if abs(g**2-d)<e:
        break
    print(g)
    g =  (g + d/g )/2
g ,g**2   


2
2.25
2.236111111111111


(2.2360679779158037, 5.000000001860471)

In [1]:
def myfunc(x):
    return x**3
f = myfunc
f(4)

64

## END